In [1]:
from lib import *
from model import *
buildAndStoreModel_lr(YELLOW, FEATURES, VARIABLE, COMMON_FETCH_COLUMNS)

sql engine ready


In [ ]:
from lib import *
from model import *
buildAndStoreModel_rfr(YELLOW, FEATURES, VARIABLE, COMMON_FETCH_COLUMNS)

sql engine ready
